In [18]:
!pip install numpy sentence-transformers groq

zsh:1: command not found: pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
import numpy as np
import os
from sentence_transformers import SentenceTransformer
from groq import Groq

In [20]:
documents = [
    "Machine Learning é um campo da inteligência artificial que permite que computadores aprendam a partir de dados.",
    "O Aprendizado de máquina dá aos sistemas a capacidade de melhorar seu desempenho sem serem explicitamente programados.",
    "Em vez de seguir apenas regras fixas, o machine learning descobre relações escondidas nos dados.",
    "Esse campo combina estatística, algoritmos e poder computacional para extrair conhecimento.",
    "O objetivo é criar modelos capazes de generalizar além dos exemplos vistos no treinamento.",
    "Aplicações de machine learning vão desde recomendações de filmes até diagnósticos médicos.",
    "Diferente de um software tradicional, o ML adapta-se conforme novos dados chegam.",
    "Os algoritmos de aprendizado de máquina transformam dados brutos em previsões úteis.",
    "O aprendizado pode ser supervisionado, não supervisionado ou por reforço, dependendo do tipo do problema.",
    "Na prática, machine learning é o motor que impulsiona muitos avanços em visão computacional e processamento de linguagem natural.",
    "Mais do que encontrar padrões, o machine learning ajuda a tomar decisões baseadas em evidências.",
    "Machine learning permite que sistemas identifiquem padrões complexos em grandes volumes de dados.",
    "Com dados suficientes, modelos de ML podem alcançar desempenho próximo ou superior ao humano.",
    "A qualidade dos dados influencia diretamente o sucesso de um modelo de aprendizado de máquina.",
    "Pré-processamento de dados é uma etapa fundamental em projetos de machine learning.",
    "Modelos de ML aprendem ajustando parâmetros internos para minimizar erros.",
    "O treinamento de um modelo envolve exemplos históricos representativos do problema.",
    "Generalização é a capacidade do modelo funcionar bem com dados nunca vistos.",
    "Overfitting ocorre quando o modelo aprende demais os dados de treino.",
    "Underfitting acontece quando o modelo é simples demais para capturar padrões relevantes.",
    "Validação cruzada é usada para avaliar a robustez de modelos de machine learning.",
    "Algoritmos supervisionados utilizam dados rotulados durante o treinamento.",
    "No aprendizado não supervisionado, os dados não possuem rótulos prévios.",
    "Clustering é uma técnica comum de aprendizado não supervisionado.",
    "Redes neurais artificiais são inspiradas no funcionamento do cérebro humano.",
    "O deep learning é um subcampo do machine learning baseado em redes profundas.",
    "Machine learning é amplamente usado em sistemas de recomendação.",
    "A classificação é uma tarefa comum em problemas de aprendizado supervisionado.",
    "Regressão é utilizada para prever valores numéricos contínuos.",
    "Modelos de ML podem ser aplicados em dados estruturados e não estruturados.",
    "A escolha do algoritmo depende do tipo de problema e dos dados disponíveis.",
    "Machine learning auxilia na automação de processos complexos.",
    "A interpretabilidade do modelo é importante em aplicações críticas.",
    "Algoritmos de ML aprendem iterativamente a partir dos erros cometidos.",
    "O desempenho de um modelo é medido por métricas específicas.",
    "Precisão e recall são métricas comuns em problemas de classificação.",
    "Machine learning é essencial para análise preditiva.",
    "Sistemas baseados em ML evoluem conforme novos dados são incorporados.",
    "Feature engineering pode melhorar significativamente os resultados do modelo.",
    "Machine learning é usado para detecção de fraudes financeiras.",
    "A escalabilidade é um desafio em sistemas de aprendizado de máquina.",
    "Modelos treinados podem ser reutilizados em diferentes contextos.",
    "Aprendizado por reforço baseia-se em recompensas e penalidades.",
    "Agentes inteligentes aprendem a agir em ambientes dinâmicos.",
    "Machine learning reduz a necessidade de regras explícitas.",
    "O balanceamento de classes é importante em bases de dados desiguais.",
    "Modelos probabilísticos são comuns em machine learning.",
    "O treinamento pode ser computacionalmente custoso.",
    "Machine learning é uma ferramenta-chave na ciência de dados.",
    "A escolha de hiperparâmetros impacta o desempenho do modelo.",
    "Pipeline de ML organiza as etapas do processo de aprendizado.",
    "Modelos simples podem ser mais eficazes que modelos complexos.",
    "Machine learning permite personalização em larga escala.",
    "A avaliação contínua garante a qualidade do modelo em produção.",
    "Dados ruidosos podem prejudicar o aprendizado.",
    "Machine learning ajuda a extrair valor de grandes bases de dados.",
    "A ética no uso de ML é uma preocupação crescente.",
    "Modelos precisam ser monitorados após a implantação.",
    "Machine learning está presente em diversas áreas do conhecimento.",
    "O ciclo de vida do ML envolve dados, modelos e decisões."
]

In [ ]:
groq_api_key = os.getenv("GROQ_API_KEY")

model = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = model.encode(documents)

client = Groq(api_key=groq_api_key)

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
def retrieve(query, top_k=3):
    query_embedding = model.encode([query])[0]

    similarities = []
    for i, doc_embedding in enumerate(doc_embeddings):
        sim = cosine_similarity(query_embedding, doc_embedding)
        similarities.append((i, sim))

    similarities.sort(key=lambda x: x[1], reverse=True)

    return [(documents[i], sim) for i, sim in similarities[:top_k]]

In [ ]:
def generate_answer(query, retrieved_docs):
    context = "\n".join([doc for doc, _ in retrieved_docs])

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {
                "role": "system",
                "content": "Você é um especialista em machine learning. Use apenas o contexto fornecido para responder."
            },
            {
                "role": "user",
                "content": f"Contexto:\n{context}\n\nPergunta: {query}"
            }
        ],
        temperature=0
    )

    return response.choices[0].message.content

In [ ]:
def rag(query, top_k=3):
    retrieved = retrieve(query, top_k)
    answer = generate_answer(query, retrieved)
    return answer, retrieved

In [ ]:
answer, docs = rag("O que é inteligencia artificial?")
print(answer)
for doc, similarity in docs:
    print(f"- {similarity:.3f}: {doc}")

A inteligência artificial (IA) é um campo da ciência da computação que visa criar sistemas capazes de realizar tarefas que normalmente requerem inteligência humana, como aprendizado, raciocínio e tomada de decisões. A IA utiliza algoritmos e técnicas de processamento de dados para permitir que os computadores aprendam a partir de experiências e melhorem suas habilidades ao longo do tempo.

No contexto do problema apresentado, a IA é fundamental para o desenvolvimento de agentes inteligentes que aprendem a agir em ambientes dinâmicos. Além disso, a IA é o campo que abrange o Machine Learning (ML), que é o foco do ciclo de vida do ML mencionado.

Em resumo, a inteligência artificial é a base para o desenvolvimento de sistemas capazes de aprender e melhorar suas habilidades, incluindo os agentes inteligentes e os modelos de Machine Learning.
- 0.652: Agentes inteligentes aprendem a agir em ambientes dinâmicos.
- 0.618: Machine Learning é um campo da inteligência artificial que permite que